In [7]:
from dotenv import load_dotenv
import os 
from openai import AzureOpenAI
import pandas as pd

In [8]:
load_dotenv()

endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
api_key = os.getenv('AZURE_OPENAI_API_KEY')

In [ ]:
%pip install pyodbc

In [ ]:
%sudo apt install unixodbc

In [52]:
import pyodbc
from sqlalchemy import create_engine
import urllib
import pandas as pd

server = os.getenv('AZURE_SQL_SERVER') 
database = os.getenv('AZURE_SQL_DB_NAME')
username = os.getenv('AZURE_SQL_USER') 
password = os.getenv('AZURE_SQL_PASSWORD') 

connection_string = f'Driver={{ODBC Driver 18 for SQL Server}};Server=tcp:{server},1433;Database={database};Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

params = urllib.parse.quote_plus(connection_string)
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine_azure = create_engine(conn_str,echo=True)

In [17]:
df = pd.read_csv('sales_data_sample.csv')
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [18]:
df.to_sql(
    'sales_data',
    engine_azure
)

2024-07-01 17:00:46,696 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-01 17:00:46,714 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-07-01 17:00:46,715 INFO sqlalchemy.engine.Engine [cached since 413.3s ago] ('BASE TABLE', 'VIEW', 'sales_data', 'dbo')
2024-07-01 17:00:46,901 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales_data (
	[index] BIGINT NULL, 
	[ORDERNUMBER] BIGINT NULL, 
	[QUANTITYORDERED] BIGINT NULL, 
	[PRICEEACH] FLOAT(53) NULL, 
	[ORDERLINENUMBER] BIGINT NULL, 
	[SALES] FLOAT(53) NULL, 
	[ORDERDATE] VARCHAR(max) NULL, 
	[STATUS] VARCHAR(max) NULL, 
	[QTR_ID] BIGINT NULL, 
	[MONTH_ID] BIGINT NU

23

In [53]:
# verify that the table was written
query = """select COLUMN_NAME, DATA_TYPE
  from information_schema.columns 
 where table_name = 'sales_data'
 order by ordinal_position"""
df2 = pd.read_sql(query, engine_azure)
df2.head()


2024-07-02 15:38:45,591 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-07-02 15:38:45,596 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-02 15:38:45,695 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-07-02 15:38:45,696 INFO sqlalchemy.engine.Engine [generated in 0.00256s] ()
2024-07-02 15:38:46,012 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-07-02 15:38:46,014 INFO sqlalchemy.engine.Engine [generated in 0.00211s] ()
2024-07-02 15:38:46,092 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2024-07-02 15:38:46,093 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ()
2024-07-02 15:38:46,352 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-02 15:38:46,354 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLE

,COLUMN_NAME,DATA_TYPE
0,index,bigint
1,ORDERNUMBER,bigint
2,QUANTITYORDERED,bigint
3,PRICEEACH,float
4,ORDERLINENUMBER,bigint


In [26]:
print(df2.to_string())

         COLUMN_NAME DATA_TYPE
0              index    bigint
1        ORDERNUMBER    bigint
2    QUANTITYORDERED    bigint
3          PRICEEACH     float
4    ORDERLINENUMBER    bigint
5              SALES     float
6          ORDERDATE   varchar
7             STATUS   varchar
8             QTR_ID    bigint
9           MONTH_ID    bigint
10           YEAR_ID    bigint
11       PRODUCTLINE   varchar
12              MSRP    bigint
13       PRODUCTCODE   varchar
14      CUSTOMERNAME   varchar
15             PHONE   varchar
16      ADDRESSLINE1   varchar
17      ADDRESSLINE2   varchar
18              CITY   varchar
19             STATE   varchar
20        POSTALCODE   varchar
21           COUNTRY   varchar
22         TERRITORY   varchar
23   CONTACTLASTNAME   varchar
24  CONTACTFIRSTNAME   varchar
25          DEALSIZE   varchar


In [30]:
import json

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version='2024-02-01'
)

deployment_name ='gpt-4o-global'

def query_azure_sql(query):
    """Run a SQL query on Azure SQL and return results as a pandas DataFrame"""
    print(f"Executing query on Azure SQL: {query}")
    df = pd.read_sql(query, engine_azure)
    return json.dumps(df.to_dict(orient='records'))


tools = [
        {
            "type": "function",
            "function": {
                "name": "query_azure_sql",
                "description": "Execute a SQL query to retrieve information from a database",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The SQL query to execute",
                        },
                    },
                    "required": ["query"],
                },
            }
        }
    ]

messages = [
    {"role":"system", "content":"""You are a helpful AI data analyst assistant, 
     You can execute SQL queries to retrieve information from a sql table, the table name is sales_data
     Here is the table schema
              COLUMN_NAME DATA_TYPE
0              index    bigint
1        ORDERNUMBER    bigint
2    QUANTITYORDERED    bigint
3          PRICEEACH     float
4    ORDERLINENUMBER    bigint
5              SALES     float
6          ORDERDATE   varchar
7             STATUS   varchar
8             QTR_ID    bigint
9           MONTH_ID    bigint
10           YEAR_ID    bigint
11       PRODUCTLINE   varchar
12              MSRP    bigint
13       PRODUCTCODE   varchar
14      CUSTOMERNAME   varchar
15             PHONE   varchar
16      ADDRESSLINE1   varchar
17      ADDRESSLINE2   varchar
18              CITY   varchar
19             STATE   varchar
20        POSTALCODE   varchar
21           COUNTRY   varchar
22         TERRITORY   varchar
23   CONTACTLASTNAME   varchar
24  CONTACTFIRSTNAME   varchar
25          DEALSIZE   varchar
     
     """},
    {"role": "user", "content": "What's the total sales revenue in the USA?"} 
]

response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    # Process the model's response
response_message = response.choices[0].message
messages.append(response_message)

print("Model's response:")  
print(response_message) 

Model's response:
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_diEjqbeb7XClJmbHA0TQs7xX', function=Function(arguments='{\n  "query": "SELECT SUM(SALES) as total_sales_revenue FROM sales_data WHERE COUNTRY = \'USA\'"\n}', name='query_azure_sql'), type='function')])


In [31]:
if response_message.tool_calls:
    for tool_call in response_message.tool_calls:
        if tool_call.function.name == "query_azure_sql":
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function arguments: {function_args}")  
            sql_response = query_azure_sql(
                    function_args.get("query")
                )
            messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "query_azure_sql",
                    "content": sql_response,
                })
else:
    print("No tool calls were made by the model.")  

    # Second API call: Get the final response from the model
final_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
    )

print(final_response.choices[0].message.content)

Function arguments: {'query': "SELECT SUM(SALES) as total_sales_revenue FROM sales_data WHERE COUNTRY = 'USA'"}
Executing query on Azure SQL: SELECT SUM(SALES) as total_sales_revenue FROM sales_data WHERE COUNTRY = 'USA'
2024-07-01 17:53:32,789 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-01 17:53:32,794 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-07-01 17:53:32,796 INFO sqlalchemy.engine.Engine [cached since 811.5s ago] ('BASE TABLE', 'VIEW', "SELECT SUM(SALES) as total_sales_revenue FROM sales_data WHERE COUNTRY = 'USA'", 'dbo')


2024-07-01 17:53:32,953 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) as total_sales_revenue FROM sales_data WHERE COUNTRY = 'USA'
2024-07-01 17:53:32,954 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-01 17:53:33,039 INFO sqlalchemy.engine.Engine ROLLBACK
The total sales revenue in the USA is $3,627,982.83.


In [51]:
import asyncio


stream = client.chat.completions.create(
        model='gpt-4o-global',
        messages=[
            {'role': 'user', 'content': "If Paris wasn't the capital of France, what city would be and why?"}
        ],
        stream=True,
        temperature=0 # this time, we set stream=True
    )


async def print_stream_chunks(stream):
    for chunk in stream:
        if chunk.choices and chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)
        await asyncio.sleep(0.1)


loop = asyncio.get_event_loop()
loop.run_until_complete(print_stream_chunks(stream))

/home/vhoudebine/miniconda3/envs/openai/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'run_conversation.<locals>.print_stream_chunks' was never awaited
  return compile(source, filename, mode, flags,


RuntimeError: This event loop is already running